In [ ]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

from sklearn import metrics

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

**Load Data:**

In [ ]:
# load training data prepared previously

df_train = pd.read_excel('/content/Final Dataset head.xlsx')
df_train.head()

,Unnamed: 0,id,cycle,setting1,setting2,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,0,1,1,0.65625,0.692308,0.670881,0.596215,0.421968,0.282214,0.343328,...,0.325581,0.152259,0.347076,0.698789,0.375,0.567450,0.289574,0.500000,0.620099,142
1,1,1,2,0.34375,0.230769,0.304560,0.182965,0.504025,0.225240,0.573590,...,0.395349,0.277907,0.227709,0.687998,0.500,0.378570,0.336740,0.645455,0.645718,141
2,2,1,3,0.53125,0.538462,0.534520,0.419558,0.464814,0.346130,0.295640,...,0.325581,0.192892,0.533557,0.457542,0.500,0.357485,0.335740,0.700000,0.681104,140
3,3,1,4,0.77500,0.461538,0.678870,0.413249,0.391587,0.449867,0.385460,...,0.372093,0.217896,0.282359,0.198790,0.250,0.227640,0.289430,0.627273,0.620382,139
4,4,1,5,0.60000,0.461538,0.308790,0.435331,0.471306,0.357974,0.285285,...,0.325581,0.187891,0.337009,0.398760,0.125,0.297490,0.189750,0.618182,0.676008,138


In [ ]:
# load test data prepared previously

df_test = pd.read_excel('/content/Final Dataset head.xlsx')
df_test.head()

,Unnamed: 0,id,cycle,setting1,setting2,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,0,1,1,0.65625,0.692308,0.670881,0.596215,0.421968,0.282214,0.343328,...,0.325581,0.152259,0.347076,0.698789,0.375,0.567450,0.289574,0.500000,0.620099,142
1,1,1,2,0.34375,0.230769,0.304560,0.182965,0.504025,0.225240,0.573590,...,0.395349,0.277907,0.227709,0.687998,0.500,0.378570,0.336740,0.645455,0.645718,141
2,2,1,3,0.53125,0.538462,0.534520,0.419558,0.464814,0.346130,0.295640,...,0.325581,0.192892,0.533557,0.457542,0.500,0.357485,0.335740,0.700000,0.681104,140
3,3,1,4,0.77500,0.461538,0.678870,0.413249,0.391587,0.449867,0.385460,...,0.372093,0.217896,0.282359,0.198790,0.250,0.227640,0.289430,0.627273,0.620382,139
4,4,1,5,0.60000,0.461538,0.308790,0.435331,0.471306,0.357974,0.285285,...,0.325581,0.187891,0.337009,0.398760,0.125,0.297490,0.189750,0.618182,0.676008,138


Create training and test data labels series.

Create features sets required to subset the original dataframes into original features and original + extracted features

In [ ]:
# original features
features_orig = ['setting1','setting2', 's1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20','s21']

# original + extracted fetures
features_extr = ['setting1', 'setting2', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']

y_train = df_train['RUL']
y_test = df_test['RUL']

**Binary Classifcation:**

Let us add some helper fuctions

In [ ]:
def bin_classify(model, clf, features, params=None, score=None):

    """Perfor Grid Search hyper parameter tuning on a classifier.

    Args:
        model (str): The model name identifier
        clf (clssifier object): The classifier to be tuned
        features (list): The set of input features names
        params (dict): Grid Search parameters
        score (str): Grid Search score

    Returns:
        Tuned Clssifier object
        dataframe of model predictions and scores

    """


    X_train = df_train[features]

    X_test = df_test[features]

    grid_search = model_selection.GridSearchCV(estimator=clf, param_grid=params, cv=5, scoring=score, n_jobs=-1)

    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)

    if hasattr(grid_search, 'predict_proba'):
        y_score = grid_search.predict_proba(X_test)[:,1]
    elif hasattr(grid_search, 'decision_function'):
        y_score = grid_search.decision_function(X_test)
    else:
        y_score = y_pred

    predictions = {'y_pred' : y_pred, 'y_score' : y_score}
    df_predictions = pd.DataFrame.from_dict(predictions)

    return grid_search.best_estimator_, df_predictions

In [ ]:
def bin_class_metrics(model, y_test, y_pred, y_score, print_out=True, plot_out=True):

    """Calculate main binary classifcation metrics, plot AUC ROC and Precision-Recall curves.

    Args:
        model (str): The model name identifier
        y_test (series): Contains the test label values
        y_pred (series): Contains the predicted values
        y_score (series): Contains the predicted scores
        print_out (bool): Print the classification metrics and thresholds values
        plot_out (bool): Plot AUC ROC, Precision-Recall, and Threshold curves

    Returns:
        dataframe: The combined metrics in single dataframe
        dataframe: ROC thresholds
        dataframe: Precision-Recall thresholds
        Plot: AUC ROC
        plot: Precision-Recall
        plot: Precision-Recall threshold; also show the number of engines predicted for maintenace per period (queue).
        plot: TPR-FPR threshold

    """

    binclass_metrics = {
                        'Accuracy' : metrics.accuracy_score(y_test, y_pred),
                        'Precision' : metrics.precision_score(y_test, y_pred),
                        'Recall' : metrics.recall_score(y_test, y_pred),
                        'F1 Score' : metrics.f1_score(y_test, y_pred),
                        'ROC AUC' : metrics.roc_auc_score(y_test, y_score)
                       }

    df_metrics = pd.DataFrame.from_dict(binclass_metrics, orient='index')
    df_metrics.columns = [model]


    fpr, tpr, thresh_roc = metrics.roc_curve(y_test, y_score)

    roc_auc = metrics.auc(fpr, tpr)

    engines_roc = []
    for thr in thresh_roc:
        engines_roc.append((y_score >= thr).mean())

    engines_roc = np.array(engines_roc)

    roc_thresh = {
                    'Threshold' : thresh_roc,
                    'TPR' : tpr,
                    'FPR' : fpr,
                    'Que' : engines_roc
                 }

    df_roc_thresh = pd.DataFrame.from_dict(roc_thresh)

    #calculate other classification metrics: TP, FP, TN, FN, TNR, FNR
    #from ground truth file, positive class = 25 => TP + FN = 25
    #from ground truth file, negative class = 75 => TN + FP = 75

    df_roc_thresh['TP'] = (25*df_roc_thresh.TPR).astype(int)
    df_roc_thresh['FP'] = (25 - (25*df_roc_thresh.TPR)).astype(int)
    df_roc_thresh['TN'] = (75*(1 - df_roc_thresh.FPR)).astype(int)
    df_roc_thresh['FN'] = (75 - (75*(1 - df_roc_thresh.FPR))).astype(int)

    df_roc_thresh['TNR'] = df_roc_thresh['TN']/(df_roc_thresh['TN'] + df_roc_thresh['FN'])
    df_roc_thresh['FNR'] = df_roc_thresh['TN']/(df_roc_thresh['TN'] + df_roc_thresh['FP'])

    df_roc_thresh['Model'] = model



    precision, recall, thresh_prc = metrics.precision_recall_curve(y_test, y_score)

    thresh_prc = np.append(thresh_prc,1)

    engines_prc = []
    for thr in thresh_prc:
        engines_prc.append((y_score >= thr).mean())

    engines_prc = np.array(engines_prc)

    prc_thresh = {
                    'Threshold' : thresh_prc,
                    'Precision' : precision,
                    'Recall' : recall,
                    'Que' : engines_prc
                 }

    df_prc_thresh = pd.DataFrame.from_dict(prc_thresh)

    if print_out:
        print('-----------------------------------------------------------')
        print(model, '\n')
        print('Confusion Matrix:')
        print(metrics.confusion_matrix(y_test, y_pred))
        print('\nClassification Report:')
        print(metrics.classification_report(y_test, y_pred))
        print('\nMetrics:')
        print(df_metrics)

        print('\nROC Thresholds:\n')
        print(df_roc_thresh[['Threshold', 'TP', 'FP', 'TN', 'FN', 'TPR', 'FPR', 'TNR','FNR', 'Que']])

        print('\nPrecision-Recall Thresholds:\n')
        print(df_prc_thresh[['Threshold', 'Precision', 'Recall', 'Que']])

    if plot_out:
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, sharex=False, sharey=False )
        fig.set_size_inches(10,10)

        ax1.plot(fpr, tpr, color='darkorange', lw=2, label='AUC = %0.2f'% roc_auc)
        ax1.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        ax1.set_xlim([-0.05, 1.0])
        ax1.set_ylim([0.0, 1.05])
        ax1.set_xlabel('False Positive Rate')
        ax1.set_ylabel('True Positive Rate')
        ax1.legend(loc="lower right", fontsize='small')

        ax2.plot(recall, precision, color='blue', lw=2, label='Precision-Recall curve')
        ax2.set_xlim([0.0, 1.0])
        ax2.set_ylim([0.0, 1.05])
        ax2.set_xlabel('Recall')
        ax2.set_ylabel('Precision')
        ax2.legend(loc="lower left", fontsize='small')

        ax3.plot(thresh_roc, fpr, color='red', lw=2, label='FPR')
        ax3.plot(thresh_roc, tpr, color='green',label='TPR')
        ax3.plot(thresh_roc, engines_roc, color='blue',label='Engines')
        ax3.set_ylim([0.0, 1.05])
        ax3.set_xlabel('Threshold')
        ax3.set_ylabel('%')
        ax3.legend(loc='upper right', fontsize='small')

        ax4.plot(thresh_prc, precision, color='red', lw=2, label='Precision')
        ax4.plot(thresh_prc, recall, color='green',label='Recall')
        ax4.plot(thresh_prc, engines_prc, color='blue',label='Engines')
        ax4.set_ylim([0.0, 1.05])
        ax4.set_xlabel('Threshold')
        ax4.set_ylabel('%')
        ax4.legend(loc='lower left', fontsize='small')

    return  df_metrics, df_roc_thresh, df_prc_thresh

Using the above functions let us model and evaluate some binary classification algorithms

In model names:

B stands for applying the model on the original features set, Before feature extraction

A stands for applying the model on the original + extracted features set, After feature extraction

In [ ]:
model = 'Logistic Regression B'
clf_lgrb = LogisticRegression(random_state=123)
gs_params = {'C': [.01, 0.1, 1.0, 10], 'solver': ['liblinear', 'lbfgs']}
gs_score = 'roc_auc'

#clf_lgrb, pred_lgrb = bin_classify(model, clf_lgrb, features_orig, params=gs_params, score=gs_score)
print('\nBest Parameters:\n',clf_lgrb)

#metrics_lgrb, roc_lgrb, prc_lgrb = bin_class_metrics(model, y_test, pred_lgrb.y_pred, pred_lgrb.y_score, print_out=True, plot_out=True)


Best Parameters:
 LogisticRegression(random_state=123)


In [ ]:
model = 'Logistic Regression A'
clf_lgra = LogisticRegression()
gs_params = {'C': [.01, 0.1, 1.0, 10], 'solver': ['liblinear', 'lbfgs']}
gs_score = 'roc_auc'

clf_lgra, pred_lgra = bin_classify(model, clf_lgra, features_extr, params=gs_params, score=gs_score)
print('\nBest Parameters:\n',clf_lgra)

metrics_lgra, roc_lgra, prc_lgra = bin_class_metrics(model, y_test, pred_lgra.y_pred, pred_lgra.y_score, print_out=True, plot_out=True)

ValueError: ignored